# Differential Privacy for Dynamic Data : Exercises

In [1]:
#Clone from GitHub: https://github.com/jleny/DifferentialPrivacy-course.git

## Introduction to Differential Privacy

**Exercise 1.** Show that the Laplace mechanism to answer a query $q: D \mapsto \mathbb R^n$, which outputs $q(d) + Lap\left(\frac{\Delta_1 q}{\epsilon}\right),$ with 
$\Delta_1 q = \max_{\text{Adj}(d,d')} \|q(d)-q(d')\|_1$,
is $\epsilon$-differentially private. 

[Hint: Start from the definition of differential privacy in terms of distributions.

**Exercise 2.** [To do in extra time, the computations can be messy.] Same exercise as in 1. for the Gaussian mechanism, which outputs $q(d) + \sigma N(0, I)$ and is $(\epsilon, \delta)$-differentially private, with $\sigma = \kappa_{\delta, \epsilon} \Delta_2 q$ and $\Delta_2 q = \max_{\text{Adj}(d,d')} \|q(d)-q(d')\|_2$.
Try to get the tightest multiplicative factor $\kappa_{\delta, \epsilon}$ possible.

**Exercise 3.** Install the following Julia package and test it.

In [2]:
#Pkg.clone("https://github.com/cipherlab-poly/PrivateDynamicData.jl.git")
#Pkg.test("PrivateDynamicData")
using PrivateDynamicData

Produce a $(\epsilon = log(2), \delta = 0.05)$-differentially private average of the following dataset $D \in \mathbb R^n$, with $n$ the number of participants. That is, the query is $q: D \mapsto \mathbb R$ such that $$q(D) = \frac{1}{n}\sum_{i=1}^n d_i.$$ Use the Gaussian mechanism for this purpose.
Assume that it is only known that the entries of $D$ are in $[0,1]$, and the adjacency is defined by changing one entry in $D$ arbitrarily.

In [3]:
srand(12345)  # reseed the RNG
ϵ = log(2); δ = 0.05
n = 100
D = rand(n)
Δ₂q = ??    
response = mean(D) + ??

LoadError: syntax: incomplete: premature end of input

**Exercise 4.**
Consider a query $q: \mathbb R^n \to \mathbb R^m$ which computes $q(d) = Ad + b$ for vector-valued datasets $d$ in $\mathbb R^n$, for $A$ a given $m \times n$ matrix and $b$ a given $m$-dimensional vector.

Produce an $\epsilon$-differentially private version of $q$.
For this, assume that each entry of $d$ is in $[0,1]$ and that the adjacency relation is defined by arbitrary variation in one entry of $d$.

Use the function ```laplaceMech```, which requires you to compute the $\ell_1$ sensitivity of the query first.

In [4]:
@doc laplaceMech

```
laplaceMech(x,f,l1sens::Real,ϵ::Real)
```

Computes a randomized version to f(x) according to the Laplace mechanism, for ϵ-differential privacy. l1sens is the l1-sensitivity of f, which must be computed and provided by the user. f must take values in R^k, for some k, i.e., return an array of k real values.


In [13]:
srand(12345678)  # reseed the RNG for debug.
ϵ = log(5)
n = 100
d = rand(n)
A = rand(5, n)/n
b = rand(5);

In [11]:
l1sens = ??

LoadError: syntax: incomplete: premature end of input

In [12]:
laplaceMech(d, x -> A*x+b, l1sens, ϵ)

LoadError: UndefVarError: l1sens not defined

**Exercise 5 (Matrix mechanism [Li & Miklau'12]).** We want to improve on the mechanism of the previous exercise. Consider the following scheme. We compute an intermediate $(\epsilon, \delta)$-differentially private results $q_1 = Md + w$, where $w$ is an appropriately scaled Gaussian noise. Suppose that $M$ has full column rank, so it has a left inverse $M^\dagger = (M^TM)^{-1}M^T$. Then output $M^\dagger q_1$.

Express the variance of the error vector made by this mechanism (when ensuring $(\epsilon, \delta)$-differential privacy), as a function of $M$.

## Working with Signals

**Exercise 6.** Redo the computations for the input and output perturbation schemes, for the simple example with sliding window averaging in the slides.

**Exercise 7.** Implement the Zero-Forcing Equalization mechanism. You can replace the spectral factorization step by a more crude approximation of the desired frequency response for the prefilter (this can be done with a least-squares fit using the Yule-Walker method, see *yulewalk* in Matlab). However, you should recompute the sensitivity of the prefilter you obtain, in order to inject a sufficient amount of noise at its output.

**Exercise 8.** Starting from the expression of the MSE for the LMMSE mechanism, verify the water-filling solution for the ideal prefilter [This is a constrained optimization problem, use Lagrange multipliers].

## Differentially Private Kalman filtering

**Exercise 9.** Consider n identical participants with scalar noisy dynamics
$$
x_{i,t+1} = a x_{i,t} + w_{i,t} \\
y_{i,t} = c x_{i,t} + v_{i,t}
$$
where $w_{i}$, $v_i$ are white Gaussian noise with covariances $\sigma_w^2$ and $\sigma_v^2$.
We want to output an differentially private estimate of $\sum_{i=1}^n x_i$. We assume that the signals $y_i$ are privacy sensitive and we take the adjacency relation as explained in the slides
$$
\|y_i-y'_i\|_2 \leq \rho, \text{ for some } i \text{ and }
y_j=y_j' \text{ if } j \neq i.
$$

Express analytically the mean squared error (MSE) for the best steady-state estimator $\hat z$, for the input perturbation mechanism and for the two-stage architecture with $D = [1, \ldots, 1]$.

**Exercise 10.** Verify the performance of the two-stage differentially private Kalman filter for the example given in the slides. Go over the following code and make sure you understand what it is doing.

In [4]:
@doc staticInputBlock_DPKF_ss

```
(D, P_val, X_val, Ω_val, M) = staticInputBlock_DPKF_ss(Ls, As, Cs, Vs, Vinvs, Winvs, k_priv, ρ,
                                                          m=size(As,1), p=size(Cs,1), r=size(Ls,1))
```

Compute an input matrix D for the two-block differentially private steady-state Kalman filter mechanism. D takes linear combinations of individual signals before privacy-preserving Gaussian noise injection.

Inputs: one matrix per individual

  * Ls: size (r,m,nusers)  (if want estimator of sum_i L_i x_i)
  * As: size (m,m,nusers)
  * Cs: size (p,m,nusers)
  * Vs: size (p,p,nusers)

-	Vinvs: inverses of Vs (computed externally for modularity/efficiency) -	Winvs: size (m,m,nusers) -	ρ: vector of size nusers, for the adjacency definition: ||yᵢ-yᵢ'||₂ <= ρᵢ for user i.

  * k_priv: proportionality constant for Gaussian privacy-preserving noise

injection. Compute it with k_priv = gaussianMechConstant(ϵ,δ) for your choice of ϵ, δ.


In [5]:
n = 10; m=2; p=2; r=1
Ls = zeros(r,m,n)
As = zeros(m,m,n)
Cs = zeros(p,m,n)
Vs = zeros(p,p,n)
Vinvs = zeros(p,p,n)
Ws = zeros(m,m,n)
Winvs = zeros(m,m,n)

for k=1:n
    Ls[:,:,k] = [1.0 0.0]
    As[:,:,k] = [-0.4 0.5; 0.4 0.7]
    Cs[:,:,k] = [1.0 0.0; 0.0 0.4]
    Ws[:,:,k] = 0.1*[1.0 0.2; 0.2 2]
    Winvs[:,:,k] = inv(Ws[:,:,k])
    Vs[:,:,k] = [0.3 0.0; 0.0 0.4]
    Vinvs[:,:,k] = inv(Vs[:,:,k])
end

k_priv = gaussianMechConstant(log(5), 0.05)
ρ = 1.0 * ones(n); ρ[1] = ρ[2] = 0.5;

In [6]:
(D, P_val, X_val, Ω_val, M) = staticInputBlock_DPKF_ss(Ls, As, Cs, Vs, Vinvs, Winvs, ρ, k_priv)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3581            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 15              
  Integer variables      : 0               

Optimizer started.
Conic interior-point optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 3581
Optimizer  - Cones             

(
[1.80754 -0.346596 … 0.891387 -0.230021; 0.329797 1.71415 … 0.217785 0.84615; 0.187724 0.955606 … -0.123669 -0.470434; -0.766618 0.151097 … 0.377762 -0.099054],

[0.860764 0.0117614 … 0.165435 -0.0171618; 0.0117614 0.77235 … -0.00218371 0.0851083; … ; 0.165435 -0.00218371 … 0.21569 0.0034668; -0.0171618 0.0851083 … 0.0034668 0.192668],

[1.28379],

[5.0503 -1.11432 … 0.197618 -0.0137331; -1.11432 1.94934 … -0.00922951 0.0442737; … ; 0.197618 -0.00922951 … 2.56578 -0.561207; -0.0137331 0.0442737 … -0.561207 1.24504],

[3.99923 0.0017499 … 1.37023 -0.149091; 0.0017499 3.9958 … 0.00326472 1.06564; … ; 1.37023 0.00326472 … 1.0 2.87596e-7; -0.149091 1.06564 … 2.87596e-7 0.999999])

In [7]:
D

4×20 Array{Float64,2}:
  1.80754   -0.346596   1.80754   …  -0.230021   0.891387  -0.230021
  0.329797   1.71415    0.329797      0.84615    0.217785   0.84615 
  0.187724   0.955606   0.187724     -0.470434  -0.123669  -0.470434
 -0.766618   0.151097  -0.766618     -0.099054   0.377762  -0.099054

In [8]:
@doc evaluateKFperf

```
evaluateKFperf(D, Ls, As, Cs, Vs, Ws, ρ, k_priv)
```

Compute the steady-state MSE of the two-block differentially private Kalman filter, with a static matrix D to combine input signals. The performance is computed by solving an algebraic Riccati equation. It corresponds to the MSE at the end of a measurement update state in the Kalman filter.

Inputs: one matrix per individual

  * D: size (q,nusers*p)  (q can be arbitrary, p = individual output signal dim.)
  * Ls: size (r,m,nusers)  (if want estimator of sum_i L_i x_i)
  * As: size (m,m,nusers)
  * Cs: size (p,m,nusers)
  * Vs: size (p,p,nusers)

-	Ws: size (m,m,nusers) -	ρ: vector of size nusers, for the adjacency definition: ||yᵢ-yᵢ'||₂ <= ρᵢ for user i.

  * k_priv: proportionality constant for Gaussian privacy-preserving noise

injection. Compute it with k_priv = gaussianMechConstant(ϵ,δ) for your choice of ϵ, δ.


In [9]:
evaluateKFperf(D, Ls, As, Cs, Vs, Ws, ρ, k_priv)

1.283790398265364

In [10]:
evaluateKFperf(eye(2*n), Ls, As, Cs, Vs, Ws, ρ, k_priv)

1.8396401424963682